In [21]:
# Import libraries
import numpy as np
import random
import json
import urllib3
import time
import polyline
import os
from dotenv import load_dotenv

Register in https://dev.routific.com/
Get tocken

In [22]:
# Load environment variables from .env file
load_dotenv()

True

In [23]:
token = os.getenv("ROUTIFIC_TOKEN")
URL = "https://api.routific.com/v1/vrp-long"

URL_job = "https://api.routific.com/jobs"
headers = {
        'Content-Type': 'application/json',
        'Authorization': f"bearer {token}"
    }

In [24]:
# IKEA depots with their coordinates
# https://www.guiagps.com/marcas-posicionadas/ikea/
depots = [
        {"id": "depot_1", "name": "San Sebastian de los Reyes", "lat": 40.54510, "lng": -3.61184},
        {"id": "depot_2", "name": "Alcorcón", "lat": 40.350370, "lng": -3.855863},
        {"id": "depot_3", "name": "Vallecas", "lat": 40.36977, "lng": -3.59670}
    ]

In [25]:
# Constraints
num_drivers = 18  # Total number of drivers. Max number in the free version
weight = 1500  # Weight capacity
volume = 1500  # Volume capacity
drivers_per_depot = 6  # Number of vehicles per depot
shift_start = "9:00"
shift_end = "18:00"

# Optimization problem setup
config = {
   "options": {
       "traffic": "slow",
       #"min_visits_per_vehicle": 15,
       "balance": True,
       #"min_vehicles": True,
       "shortest_distance": True,
       # "squash_durations": 1,
       "max_vehicle_overtime": 30,
       #"max_visit_lateness": 15,
       "polylines": True
   }
}

# Time to wait before checking the status again
waiting_time = 10 # seconds

In [26]:
# Function to read json with orders
def read_json(file_path):
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except Exception as e:
        print(f"Failed to read JSON from {file_path}: {e}")
        return None

In [27]:
# Function to write JSON with visits in the required format
# Things to change:
# Priority is random
# Weights and volumes are random
def format_to_visits(data):
    if data is None:
        print("Data is None, cannot format visits.")
        return {}
    
    priorities = ["low", "regular", "high"]  # List of possible priorities
    
    visits = {
        item['order_id']: {
            "location": {
                "name": item["location"].get("address", "Unknown") if isinstance(item["location"].get("address"), str) else "Unknown",
                "lat": round(item["location"]["lat"], 6),
                "lng": round(item["location"]["lon"], 7)  # Correcting 'lon' to 'lng' for consistency
            },
            "start": item.get("start", "9:00"),  # Default start time if not provided
            "end": item.get("end", "18:00"),  # Default end time if not provided
            "duration": item.get("duration", 5),  # Default duration if not provided
            "load": {
                "weight": int(np.ceil(item.get("order", {}).get("weight", 10))),  # Default weight if not provided
                "volume": int(np.ceil(item.get("order", {}).get("volume", 200)))  # Default volume if not provided
            },
            "priority": random.choice(priorities),  # Assign random priority
        } for item in data
    }
    return visits

In [28]:
def build_fleet(depots, num_drivers, shift_start, shift_end, weight, volume, drivers_per_depot):
    fleet = {
        f"driver_{driver_counter}": {
            "start_location": {
                "id": depot["id"],
                "name": depot["name"],
                "lat": depot["lat"],
                "lng": depot["lng"]
            },
            "end_location": {
                "id": depot["id"],
                "name": depot["name"],
                "lat": depot["lat"],
                "lng": depot["lng"]
            },
            "shift_start": shift_start,
            "shift_end": shift_end,
            "min_visits": 1,
            "capacity": {
                "weight": weight,
                "volume": volume
            },
            "type": ["A", "B"],
            "strict_start": True,
            "breaks": [
                {"id": "lunch", "start": "12:00", "end": "13:00"},
                {"id": "client call", "start": "15:00", "end": "15:30", "in_transit": True}
            ]
        }
        for depot in depots
        for driver_counter in range(1, num_drivers + 1)
        if driver_counter <= num_drivers and (driver_counter - 1) // drivers_per_depot < len(depots)
    }
    
    return fleet

In [29]:
def check_job_status(URL, jobID, headers, waiting):
    
    http = urllib3.PoolManager()
    URL = f"{URL}/{jobID}"
    job_status = None

    while job_status != 'finished':
        response = http.request('GET', URL, headers=headers)
        solution_data = json.loads(response.data.decode('utf-8'))
        job_status = solution_data['status']

        print("Current job status:", job_status, "...")

        if job_status in ['pending', 'processing']:
            time.sleep(waiting)
        elif job_status == 'finished':
            print("Job finished.")
            break
        else:
            print("Unexpected job status:", job_status)
            break

In [30]:
# Iterate over the response and print the key-value structure
def print_key_value_structure(data, indent=''):
    for key, value in data.items():
        print(f"{indent}Key: {key} - Value Type: {type(value)}")
        if isinstance(value, dict):
            print_key_value_structure(value, indent + '  ')

In [31]:
# Function to update the solution data and change 'location_id' to 'order_id' and delete 'polylines'
def update_solution_keys(solution_data):
    try:
        solution = solution_data['output']['solution']
        
        # Use list comprehension to update the keys
        solution = {
            driver: [
                {**stop, 'order_id': stop.pop('location_id')} if 'location_id' in stop and not stop['location_id'].startswith('depot_') else stop
                for stop in route
            ]
            for driver, route in solution.items()
        }
        
        solution_data['output']['solution'] = solution
        
        # Remove 'pl_precision' and 'polylines' keys if they exist
        solution_data['output'].pop('pl_precision', None)
        solution_data['output'].pop('polylines', None)
        
        return solution_data
    
    except KeyError as e:
        print(f"Key error: {e}")
        return solution_data

In [32]:
# Extract the information from the file
# TO BE changed to the json received from upstream
to_be_served = read_json('/Users/borja/Documents/Somniumrema/projects/de/route_optimizer/data/clients_final.json')

# Create visits file with orders information
visits = format_to_visits(to_be_served)

# Build the fleet with the number of drivers, weight, volume and vehicles per depot
fleet = build_fleet(depots, num_drivers, shift_start, shift_end, weight, volume, drivers_per_depot)

# Print to check formats
#print(json.dumps(visits, indent=4))

# Print to check formats
#print(json.dumps(fleet, indent=4))

In [33]:
# Data payload
payload = {"visits": visits, "fleet": fleet}

# Combine data and config
combined_data = {**payload, **config}

# Convert combined data to JSON string
body = json.dumps(combined_data)


In [34]:
# Create a PoolManager instance
http = urllib3.PoolManager()

# Make the POST request
req_job = http.request('POST', URL, body=body, headers=headers)

# Print the response status
print("Response status:", req_job.status)

# Obtain the response data
response_data = json.loads(req_job.data.decode('utf-8'))

# Store the jobID
jobID = response_data["job_id"]

#Print JobID
print("JobID:",jobID)

Response status: 202
JobID: m1flsw57826


In [35]:
check_job_status(URL_job, jobID, headers, waiting=waiting_time)

Current job status: processing ...
Current job status: processing ...
Current job status: processing ...
Current job status: processing ...
Current job status: processing ...
Current job status: processing ...
Current job status: processing ...
Current job status: processing ...
Current job status: processing ...
Current job status: processing ...
Current job status: finished ...
Job finished.


In [36]:
# The URL with the ID inserted
url_solution = f"{URL_job}/{jobID}"

# Make the GET request
solution = http.request('GET',url_solution,headers=headers)

# Decode response.data (bytes) to a str and then load into a dict
solution_data = json.loads(solution.data.decode('utf-8'))

# Print structure of the response_data
print_key_value_structure(solution_data)

Key: timing - Value Type: <class 'dict'>
  Key: startedProcessingAt - Value Type: <class 'str'>
  Key: finishedProcessingAt - Value Type: <class 'str'>
Key: fetchedCount - Value Type: <class 'int'>
Key: apiMajorVersion - Value Type: <class 'int'>
Key: apiMinorVersion - Value Type: <class 'int'>
Key: _id - Value Type: <class 'str'>
Key: input - Value Type: <class 'dict'>
  Key: visits - Value Type: <class 'dict'>
    Key: d823a7eb-8e22-454d-a3ea-5ad5f2348993 - Value Type: <class 'dict'>
      Key: location - Value Type: <class 'dict'>
        Key: name - Value Type: <class 'str'>
        Key: lat - Value Type: <class 'float'>
        Key: lng - Value Type: <class 'float'>
      Key: start - Value Type: <class 'str'>
      Key: end - Value Type: <class 'str'>
      Key: duration - Value Type: <class 'int'>
      Key: load - Value Type: <class 'dict'>
        Key: weight - Value Type: <class 'int'>
        Key: volume - Value Type: <class 'int'>
      Key: priority - Value Type: <class 'i

In [37]:
# Print the output from the response_data
solution_data['output']

{'total_travel_time': 4925,
 'total_idle_time': 23,
 'total_visit_lateness': 0,
 'total_vehicle_overtime': 0,
 'vehicle_overtime': {'driver_1': 0,
  'driver_10': 0,
  'driver_11': 0,
  'driver_12': 0,
  'driver_13': 0,
  'driver_14': 0,
  'driver_15': 0,
  'driver_16': 0,
  'driver_17': 0,
  'driver_18': 0,
  'driver_2': 0,
  'driver_3': 0,
  'driver_4': 0,
  'driver_5': 0,
  'driver_6': 0,
  'driver_7': 0,
  'driver_8': 0,
  'driver_9': 0},
 'total_break_time': 1590,
 'num_unserved': 0,
 'unserved': None,
 'solution': {'driver_1': [{'location_id': 'depot_3',
    'location_name': 'Vallecas',
    'arrival_time': '09:00',
    'distance': 0},
   {'location_id': 'c8ceef8c-627b-42ff-a0f0-8364e04a2be3',
    'location_name': 'Calle de Candilejas22',
    'arrival_time': '09:25',
    'finish_time': '09:30',
    'distance': 9276.9},
   {'location_id': 'abb2c2b1-1066-4e06-a65e-47647f1876c3',
    'location_name': 'Pasarela Parque Manzanares Sur',
    'arrival_time': '09:45',
    'finish_time': '09

In [38]:
# Define the file path where the solution will be stored
file_path = '/Users/borja/Documents/Somniumrema/projects/de/route_optimizer/data/raw_solution.json'

# Write the solution to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(solution_data, json_file, indent=4)

print(f"Raw solution stored in {file_path}")

Raw solution stored in /Users/borja/Documents/Somniumrema/projects/de/route_optimizer/data/raw_solution.json


In [39]:
# Data for reporting
reporting_data = update_solution_keys(solution_data)

# Show the reporting data
print(json.dumps(reporting_data, indent=4))

{
    "timing": {
        "startedProcessingAt": "2024-09-23T22:53:25.554Z",
        "finishedProcessingAt": "2024-09-23T22:55:12.971Z"
    },
    "fetchedCount": 1,
    "apiMajorVersion": 1,
    "apiMinorVersion": 0,
    "_id": "66f1f165665dd5001badc191",
    "input": {
        "visits": {
            "d823a7eb-8e22-454d-a3ea-5ad5f2348993": {
                "location": {
                    "name": "Bypass Sur",
                    "lat": 40.390131,
                    "lng": -3.6835189
                },
                "start": "09:00",
                "end": "18:00",
                "duration": 5,
                "load": {
                    "weight": 46,
                    "volume": 93
                },
                "priority": 1,
                "time_windows": [
                    {
                        "start": "09:00",
                        "end": "18:00"
                    }
                ],
                "typedDemand": {
                    "weight": 46,
  